# БПМ-19-3, Максаков Владимир, вариант 48

In [ ]:
!pip install crc64iso

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from typing import List
from math import log2, ceil
from random import randrange
from crc64iso.crc64iso import crc64

In [ ]:
def __hamming_common(src: List[List[int]], s_num: int, encode=True) -> int:
    s_range = range(s_num)
    errors = 0
    for i in src:
        sindrome = 0
        for s in s_range:
            sind = 0
            for p in range(2 ** s, len(i) + 1, 2 ** (s + 1)):
                for j in range(2 ** s):
                    if (p + j) > len(i):
                        break
                    sind ^= i[p + j - 1]
            if encode:
                i[2 ** s - 1] = sind
            else:
                sindrome += (2 ** s * sind)
        if (not encode) and sindrome:
            try:
                i[sindrome - 1] = int(not i[sindrome - 1])
            except IndexError:
                errors += 1
    return errors

In [ ]:
def hamming_encode(msg: str, mode: int = 8) -> str:
    """
    Encoding the message with Hamming code.
    :param msg: Message string to encode
    :param mode: number of significant bits
    :return: 
    """

    result = ""

    msg_b = msg.encode("utf8")
    s_num = ceil(log2(log2(mode + 1) + mode + 1))   # number of control bits
    bit_seq = []
    for byte in msg_b:  # get bytes to binary values; every bits store to sublist
        bit_seq += list(map(int, f"{byte:08b}"))

    res_len = ceil((len(msg_b) * 8) / mode)     # length of result (bytes)
    bit_seq += [0] * (res_len * mode - len(bit_seq))    # filling zeros

    to_hamming = []

    for i in range(res_len):    # insert control bits into specified positions
        code = bit_seq[i * mode:i * mode + mode]
        for j in range(s_num):
            code.insert(2 ** j - 1, 0)
        to_hamming.append(code)

    errors = __hamming_common(to_hamming, s_num, True)   # process

    for i in to_hamming:
        result += "".join(map(str, i))

    return result

In [ ]:
def hamming_decode(msg: str, mode: int = 8):
    """
    Decoding the message with Hamming code.
    :param msg: Message string to decode
    :param mode: number of significant bits
    :return: 
    """

    result = ""

    s_num = ceil(log2(log2(mode + 1) + mode + 1))   # number of control bits
    res_len = len(msg) // (mode + s_num)    # length of result (bytes)
    code_len = mode + s_num     # length of one code sequence

    to_hamming = []

    for i in range(res_len):    # convert binary-like string to int-list
        code = list(map(int, msg[i * code_len:i * code_len + code_len]))
        to_hamming.append(code)

    errors = __hamming_common(to_hamming, s_num, False)  # process

    for i in to_hamming:    # delete control bits
        for j in range(s_num):
            i.pop(2 ** j - 1 - j)
        result += "".join(map(str, i))

    msg_l = []

    for i in range(len(result) // 8):   # convert from binary-sring value to integer
        val = "".join(result[i * 8:i * 8 + 8])
        msg_l.append(int(val, 2))

    # finally decode to a regular string
    try:
        result = bytes(msg_l).decode("utf-8")
    except UnicodeDecodeError:
        pass

    return result, errors

In [ ]:
def noizer(msg: str, mode: int) -> str:
    """
    Generates an error in each element of a Hamming encoded message
    """
    seq = list(map(int, msg))
    s_num = ceil(log2(log2(mode + 1) + mode + 1))  # количество служебных битов
    code_len = mode + s_num  # длина кодового слова
    cnt = len(msg) // code_len
    result = ""

    for i in range(cnt):
        to_noize = seq[i * code_len:i * code_len + code_len]
        noize = randrange(code_len)
        to_noize[noize] = int(not to_noize[noize])
        result += "".join(map(str, to_noize))

    return result

In [ ]:
def noizer4(msg: str, mode: int) -> str:
    """
    Generates up to 4 errors in each element of a Hamming encoded message
    """
    seq = list(map(int, msg))
    s_num = ceil(log2(log2(mode + 1) + mode + 1))  # количество служебных битов
    code_len = mode + s_num  # длина кодового слова
    cnt = len(msg) // code_len
    result = ""

    for i in range(cnt):
        to_noize = seq[i * code_len:i * code_len + code_len]
        noize1 = randrange(code_len)
        noize2 = randrange(code_len)
        noize3 = randrange(code_len)
        noize4 = randrange(code_len)
        to_noize[noize1] = int(not to_noize[noize1])
        to_noize[noize2] = int(not to_noize[noize2])
        to_noize[noize3] = int(not to_noize[noize3])
        to_noize[noize4] = int(not to_noize[noize4])
        result += "".join(map(str, to_noize))

    return result

In [ ]:
MODE = 114  # длина слова с контрольными битами составляет 121 => значащих битов в слове 114
msg = "На место Рота взяли Маттиаса Ябса, но он временно был вынужден покинуть группу — Михаэль Шенкер рассорился с UFO и вернулся в Scorpions.\n Его гитара звучит в трёх композициях в альбоме Lovedrive, он участвовал и в туре, в котором группа прокатывала этот альбом. Но в 1979 году Михаэль окончательно покинул Scorpions, основав свою группу MSG, и на место соло-гитариста Scorpions окончательно вернулся Ябс.\n Альбом Lovedrive стал настоящим прорывом для группы, именно с его выходом к Scorpions пришёл международный успех, в том числе и в Америке.[16]\n В 1980 году группа записывает свой 7-й по счёту альбом — Animal Magnetism. Альбом получился очень «живым» — в лучших традициях хард-рока того времени, — и по сей день является одной из «визиток» Scorpions.\n В период с 1980 по 1981 группа много гастролировала. У Клауса Майне начинаются серьёзные проблемы с горлом[18], и он переносит операцию на голосовых связках.\n Новый альбом Blackout получает платину в США и Канаде[19][20] и попадает в Top 10 журнала Billboard, как и следующий альбом Love at First Sting — именно в этот альбом и вошла известная баллада «Still Loving You».\n По словам Майне, гастроли по Америке в 80-х были «одной непрекращающейся вечеринкой», однако при этом ни у кого из участников группы не было проблем с употреблением наркотиков. На US Festival в Калифорнии в 1983 году хэдлайнер фестиваля Van Halen, перед выступлением которых должны были выступать Scorpions, запретил использовать на сцене какие-либо спецэффекты; Scorpions нашла чем ответить, наняв 5 истребителей, которые летали над аудиторией и долиной Сан-Бернардино.\n В 1984 году Scorpions трижды собирали аншлаг в Мэдисон-Сквер-Гарден[16]. Концертный альбом World Wide Live 1985 года демонстрирует всё лучшее, чего достигла группа в течение второй фазы своей карьеры. Качественно записанный и спродюсированный, он в течение четырёх месяцев занимал 14 место в американских чартах. Перед выпуском Savage Amusement в 1988 году — их первого студийного альбома за четыре года — группа взяла заслуженный отпуск. Альбом имел огромный успех, достиг пятой позиции в США и первых в европейских чартах. В 1988 году группа впервые посещает СССР и даёт 10 аншлаговых концертов в Ленинграде (а также спонтанно выступает в Ленинградском рок-клубе)[21][22][23]. В 1989 году группа перешла на сотрудничество с Phonogram Records и тем завершила 15-летнюю работу с продюсером Дитером Дирксом. В августе того же года Scorpions с успехом выступают на Московском международном фестивале мира, став одной из первых западных рок-групп, давших концерт в СССР[24]."
print(f'Сообщение:\n{msg}')
checksum = crc64(msg)
print(f'Контрольная сумма: {checksum}')

# Первая отправка (без ошибок)
print('-----------ПЕРВАЯ ОТПРАВКА-----------')
enc_msg = hamming_encode(msg, MODE)
print(f'Кодированное сообщение:\n{enc_msg}')
dec_msg, err = hamming_decode(enc_msg, MODE)
dec_msg = dec_msg[:-2:]
print(f'Раскодированное сообщение:\n{dec_msg}')
print(
    f'Контрольная сумма: {crc64(dec_msg)}, корректность: {crc64(dec_msg) == checksum}')
print(f'MSG: {msg == dec_msg}')

# Вторая отправка (не более 1 ошибки на слово)
print('-----------ВТОРАЯ ОТПРАВКА-----------')
noize_msg = noizer(enc_msg, MODE)
print(f'Кодированное сообщение с ошибками:\n{noize_msg}')
dec_msg, err = hamming_decode(noize_msg, MODE)
dec_msg = dec_msg[:-2:]
print(f'Раскодированное сообщение:\n{dec_msg}')
print(
    f'Контрольная сумма: {crc64(dec_msg)}, корректность: {crc64(dec_msg) == checksum}')
print(f'MSG: {msg == dec_msg}')

# Третья отправка (4 ошибки на слово)
print('-----------ТРЕТЬЯ ОТПРАВКА-----------')
noize_msg = noizer4(enc_msg, MODE)
print(f'Кодированное сообщение с ошибками:\n{noize_msg}')
dec_msg, err = hamming_decode(noize_msg, MODE)
dec_msg = dec_msg[:-2:]
print(f'Раскодированное сообщение:\n{dec_msg}')
print(
    f'Контрольная сумма: {crc64(dec_msg)}, корректность: {crc64(dec_msg) == checksum}, количество обнаруженных ошибок: {err}')

Сообщение:
На место Рота взяли Маттиаса Ябса, но он временно был вынужден покинуть группу — Михаэль Шенкер рассорился с UFO и вернулся в Scorpions.
 Его гитара звучит в трёх композициях в альбоме Lovedrive, он участвовал и в туре, в котором группа прокатывала этот альбом. Но в 1979 году Михаэль окончательно покинул Scorpions, основав свою группу MSG, и на место соло-гитариста Scorpions окончательно вернулся Ябс.
 Альбом Lovedrive стал настоящим прорывом для группы, именно с его выходом к Scorpions пришёл международный успех, в том числе и в Америке.[16]
 В 1980 году группа записывает свой 7-й по счёту альбом — Animal Magnetism. Альбом получился очень «живым» — в лучших традициях хард-рока того времени, — и по сей день является одной из «визиток» Scorpions.
 В период с 1980 по 1981 группа много гастролировала. У Клауса Майне начинаются серьёзные проблемы с горлом[18], и он переносит операцию на голосовых связках.
 Новый альбом Blackout получает платину в США и Канаде[19][20] и попадает 